In [122]:
from dotenv import load_dotenv
import time, os
from dotenv import load_dotenv

load_dotenv()
USERNAME = os.environ.get('USERNAME')
PASSWORD = os.environ.get('PASSWORD')

In [123]:
PATH = r'chromedriver.exe'

In [124]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--start-maximized")  # Maximize browser window
service = Service(PATH)
driver = webdriver.Chrome(service=service, options=options)

In [125]:
url = 'https://www.instagram.com/accounts/login/'
driver.get(url)

In [126]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
username=WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.NAME, "username"))
)
username.send_keys(USERNAME)

password=WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.NAME,"password"))
)

password.send_keys(PASSWORD)
time.sleep(20)
submit=WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.XPATH,"/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div/section/main/div/div/div[1]/div[2]/div/form/div/div[3]/button"))
)
time.sleep(3)
submit.click()

In [173]:
url = 'https://www.instagram.com/p/DBwQwNrNgGl/?igsh=MTAycXoyd3QzdTZvdA=='
driver.get(url)

In [35]:
scrollable_element = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]')

# Initialize last height
last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

# Scroll the element to load more comments
while True:
    # Scroll to the bottom of the scrollable element
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)
    time.sleep(5)  # Allow time for new comments to load

    # Get the new scroll height
    new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    # Break the loop if no new comments are loaded
    if new_height == last_height:
        break
    last_height = new_height

In [174]:
import re
time_pattern = re.compile(r"^\d+[dwhm]$")
view_replies = re.compile(r"^View all (\d+)? replies")

In [175]:
time.sleep(25)
comments_container = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[2]")
# Now find all individual comments within the container
print("Done scrolling...")



username = comments_container.find_elements(By.XPATH, ".//span[contains(@class, '_ap3a _aaco _aacw _aacx _aad7 _aade')]")
usernames = []
# Extract and print the text of the filtered elements
for comment in username:
    usernames.append(comment.text)

comment = comments_container.find_elements(By.TAG_NAME, "span")

actual_comments = []



for element in comment:
    text = element.text.strip()  # Get the text and remove extra spaces
    # Exclude elements with irrelevant content
    if text and "like" not in text.lower() and "reply" not in text.lower() and "see translation" not in text.lower() and text not in usernames and not time_pattern.match(text) and not view_replies.match(text):
        actual_comments.append(text)

# Print actual comments
for comment in actual_comments:
    print(comment)


In [177]:
import csv

# Open a CSV file for writing
with open('instagram_comments.csv', mode='a', newline='', encoding='utf-8') as file:
    # Define the column headers
    writer = csv.writer(file)
    #writer.writerow(['Comment'])

    for comments in actual_comments:
        Comment = comments

        row_data = [Comment]
        writer.writerow(row_data)
    

In [267]:
import pandas as pd
df = pd.read_csv('instagram_comments.csv')

Check for blanks

In [ ]:
df[df['Comment']==""]

,Comment


In [259]:
df = df[df['Comment']!=""]

In [181]:
import re
emoji_pattern = re.compile('[\U0001F600-\U0001F64F'
                           '\U0001F300-\U0001F5FF'
                           '\U0001F680-\U0001F6FF'
                           '\U0001F700-\U0001F77F'
                           '\U0001F780-\U0001F7FF'
                           '\U0001F800-\U0001F8FF'
                           '\U0001F900-\U0001F9FF'
                           '\U0001FA00-\U0001FA6F'
                           '\U0001FA70-\U0001FAFF'
                           '\U00002702-\U000027B0'
                           '\U000024C2-\U0001F251]+')

# Function to extract emojis from a string
def delete_emojis(text):
    return emoji_pattern.sub('', str(text))

In [182]:
df['Comment'] = df['Comment'].apply(delete_emojis)

In [253]:
def remove_mentions(cell):
    return ' '.join(word for word in str(cell).split() if not word.startswith('@'))

# Apply the function to the entire DataFrame
df = df.map(remove_mentions)

In [256]:
def remove_hashtag(cell):
    return ' '.join(word.lstrip('#') if word.startswith('#') else word for word in str(cell).split())

# Apply the function to the entire DataFrame
df = df.map(remove_hashtag)

In [257]:
def contains_mention(cell):
    return any(word.startswith('#') for word in str(cell).split())

# Filter rows where any cell contains a word starting with '@'
df[df.map(contains_mention).any(axis=1)]

,Comment


In [263]:
df.to_csv('instagram_comments.csv', index=False)